In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
import random
import re
import pickle
from datasets import load_dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def mask_spans(text, mask_token="[mask]", mask_prob=0.15):
    if len(text) == 0:
        return None
    text = text[0]
    words = text.split()
    n = len(words)
    if n < 10 or n > 20:
        return None
    num_masks = random.sample(range(1,5), 1)[0]
    mask_index = random.sample(range(1,n-num_masks-1), 1)[0]

    masked_text = words[:]
    targets = []

    masked_text[mask_index] = mask_token
    targets.append(words[mask_index])
    for i in range(1, num_masks):
        targets.append(masked_text[mask_index+1])
        masked_text.pop(mask_index+1)

    masked_text = " ".join(masked_text)
    targets = " [answer] ".join(targets)

    return masked_text, text

In [6]:
def create_training_example(text):
    out = mask_spans(text)
    if out == None:
      return None
    training_example = f"{out[0]} [sep] {out[1]}"
    return training_example

In [7]:
dataset = load_dataset("embedding-data/sentence-compression")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/180000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 180000
    })
})


In [11]:
train_corpus = dataset["train"]["set"][:170000]
train_corpus = [create_training_example(text) for text in train_corpus]
train_corpus = [x for x in train_corpus if x is not None]
print(train_corpus[0:100])

with open('/content/drive/MyDrive/UCLA/CS263/Proj/train_data.pkl', 'wb') as file:
    pickle.dump(train_corpus, file)

['Major League [mask] will be speaking at St. Norbert College next month. [sep] Major League Baseball Commissioner Bud Selig will be speaking at St. Norbert College next month.', "It's fresh cherry time in [mask] best time to enjoy this delicious and nutritious fruit. [sep] It's fresh cherry time in Michigan and the best time to enjoy this delicious and nutritious fruit.", 'THE JSE kept [mask] all time high by midday today as resources continued to fuel the bourse. [sep] THE JSE kept toying with an all time high by midday today as resources continued to fuel the bourse.', 'The government is defending the latest [mask] statistics despite a worrying rise in the recorded amount of violent offending. [sep] The government is defending the latest police crime statistics despite a worrying rise in the recorded amount of violent offending.', 'The renovated Marappalam [mask] been opened for two-wheelers last week, was opened for other vehicles also on Friday. [sep] The renovated Marappalam brid

In [12]:
test_corpus = dataset["train"]["set"][170000:]
test_corpus = [create_training_example(text) for text in test_corpus]
test_corpus = [x for x in test_corpus if x is not None]
test_corpus = test_corpus[2:22]
print(test_corpus)

with open('/content/drive/MyDrive/UCLA/CS263/Proj/test_data.pkl', 'wb') as file:
    pickle.dump(test_corpus, file)

['The NBA officially released its preseason schedule today, complete [mask] sent out to the media. [sep] The NBA officially released its preseason schedule today, complete with a press release sent out to the media.', 'French government officials and citizens have embraced Burmese [mask] Suu Kyi, who is within days of ... [sep] French government officials and citizens have embraced Burmese democracy advocate Aung San Suu Kyi, who is within days of ...', "India and Sri Lanka today agreed not to resort to violence against fishermen straying [mask] maritime boundary. [sep] India and Sri Lanka today agreed not to resort to violence against fishermen straying into each-other's maritime boundary.", 'Foreclosure rates are holding steady in the Fort Smith region [mask] a year ago. [sep] Foreclosure rates are holding steady in the Fort Smith region compared to a year ago.', 'Police say a 17-year-old juvenile was arrested for [mask] home in Dover on Thursday. [sep] Police say a 17-year-old juven